---
# Syntactic Sugar

# What is Syntactic Sugar?
Sometimes used as  a derogatory term by stuffy types, **syntactic sugar** refers to features of a programming language that make more verbose actions quicker and slicker. You may have already been using one of these techniques, known as **augmented assignment**.

Compare the following two cells, which do *exactly* the same thing. You'll explore this feature in more detail in your lab.

In [ ]:
x = 1
x = x + 2
x

In [ ]:
x = 1
x += 2
x

# List Comprehensions: Creating Lists from Tuples
We've seen many times when we want to start from an iterable containing data (suppose a bunch of $x$ values), and wish to create a new iterable using those values (suppose, some function evaluated at all of those $x$ values). We've seen that pylab/numpy arrays can make this a lot simpler, but for non-numerical data or more complicated operations that cannot be vectorized.

For simple operations, a **list comprehension** can be an efficient way to quickly synthesize a list from another iterable. Consider the two following equivalent cells.

In [ ]:
squares = []
for i in range(10):
    squares.append(i**2)
squares

In [ ]:
squares = [i**2 for i in range(10)]
squares

# Conditions for List Comprehensions
- Must have a source iterable (`for x in source` at the end of the comprehension)
- Synthesized list will have the same length as the source iterable
- Each element of the list to be generated must be synthesized in a **single line** of code using an element of the source iterable.

# Additional Feature: Filtering in List Comprehensions
If you only want the comprehension to "fire" off for certain input values, you can append an `if` test to the end of the comprehension. So to get the squares of multiples of 3 less than 20, we could do the following:

In [ ]:
[x**2 for x in range(20) if x % 3 == 0]

In fact, the filtering feature is so nifty that you can create some silly list comprehensions that do nothing to the input but filter it:

In [ ]:
# get numbers that are 3 more than a multiple of 4: x = 4n + 3 for some integer n
[x for x in range(50) if (x - 3) % 4 == 0]

# Example Where List Comprehensions Won't Work
Suppose we have a composition of functions that are each defined piecwise, like the following:
$$f(x) = \begin{cases}x > 0: & \cos x \\ x \leq 0: & \exp(-x)\end{cases}$$
$$g(x) = \begin{cases}x > \frac{1}{2}: & x^2 \\  x \leq \frac{1}{2}: & x^3\end{cases}$$
$$h(x) = g \circ f(x) = g(f(x))$$
Starting with an iterable of $x$ values, it is impossible to convert it to a list of values obtained from passing those $x$ values into $h$ with a list comprehension.

In [ ]:
from math import cos, exp
xs = list(range(-10, 11))
ys = []
for x in xs:
    # no way to reduce this junk to a single line
    if x > 0:
        f = cos(x)
    else:
        f = exp(-x)
    if f > 1/2:
        ys.append(f**2)
    else:
        ys.append(f**3)

# Lambdas are like diet functions
Similar to how list comprehensions work as a single line for-loop that generates a new loop, a lambda is a single-line function. It doesn't even have to have a name associated with it!

The following two cells do the same thing.

In [ ]:
def squared_plus_one(x):
    return x**2 + 1

squared_plus_one(3)

In [ ]:
squared_plus_one = lambda x: x**2  + 1
squared_plus_one(3)

# Anatomy of a lambda
```python
squared_plus_one = lambda x: x**2  + 1
```
- The name `squared_plus_one` is now a function
- The `lambda` keyword starts the lambda. Immediately after it, we can given an arbitrary number of input variables.
- After the list of input variables, we add a colon
- After the colon is a single expression that is implicitly the `return` value of the function
- **Can only have one line**

# Why use lambdas?
- You never really *need* to use them
- A bit quicker than a multi-line defintion for a quick one-line function that is self-explanatory
- Very handy when you just need to pass one function in as an argument to another function, but don't really need it otherwise (next example)

# Example: Numerical Differentiation
Below we create a function that computes a numerical derivative essentially by "taking the limit" of $\frac{\Delta f}{\Delta x}$ over smaller and smaller windows of $\Delta x$ until the value stops changing. The first parameter of this function *is itself a function*. Below we show usage of this function with a lambda, so that we define it right there in the call to `diff`; no need to set it up as a named function earlier.

In [ ]:
def diff(f, x0, tol=1e-5):
    """Evaluate numerical derivative of `f` at `x0`."""
    dx = 1
    dfdx = (f(x0 + dx) - f(x0 - dx)) / (2 * dx)
    diff = 2 * tol
    while diff > tol:
        dx /= 2
        dfdx_old = dfdx
        dfdx = (f(x0 + dx) - f(x0 - dx)) / (2 * dx)
        diff = abs(dfdx - dfdx_old)
    return dfdx

# find derivative of x^2 + 3x - 1 evaluated at 5
diff(lambda x: x**2 + 3*x - 1, 5)

# A Quickie: Files and Contexts
Whenever we open a file, we know we want to close it. The file object itself is basically ephemeral; once it is closed, it's no longer useful. Python offers a syntax that makes this more explicit, and it takes care of closing the file for us. Consider the two cells below, which do the same thing (with different file names).

In [ ]:
# the old way (totally fine)
f = open('testing.txt', 'w')
f.write("I'm writing to to a file.")
f.close()

In [ ]:
# the cleaner way
with open('testing2.txt', 'w') as f:
    f.write("I'm writing to a file")

The `with`...`as`...`:` setup is called a "context", and it shows up elsewhere in python land, usually when there is some implicit "setup" and "cleanup" process that needs to be done on either side of a block of code.

# `map` and `filter`: More ways to generate iterables from iterables

The `map` function takes in a function of one variable and an iterable, applies the provided function to every element of the iterable, and returns the resulting iterable. In this way, it is very similar to a list comprehension, **except** it does not return a list. It returns a `map` object, which computes the elements as they are needed, rather than right when it was instantiated.

# Example: A trillion multiples of 3: list comprehensions vs `map`
`map` in action: it executes immediately

In [ ]:
mult_3_map = map(lambda x: 3 * x, range(1_000_000_000_000))

And now with a list comprehension (it won't work; you'll have to stop the execution because it'll get bogged down)

In [ ]:
mult_3_comp = [3 * x for x in range(1_000_000_000_000)]

Why The difference? It's because `map` sets up an iterable that **knows how to calculate** each of its values as they are requested in another `for` loop, but it doesn't compute them all at once. You can force this behavior by casting it to a list, but at that point, you may as well have used a lit comprehension.

# `filter`: Grab only the elements you want from an iterable
Simliar to `map`, `filter` takes a function and an iterable, but in this case, the function **must** return either `True` or `False`. It then returns an iterable containing only the values that cause the function to return `True`, thus filtering out the input iterable.

This is again very similar to the `if` feature in list comprehension, but just like with `map`, the `filter` iterable doesn't hold all of its objects at once; it only grabs them as they are needed.

# Example: Multiples of 3 under a trillion
`filter` in action: it executes immediately

In [ ]:
mult_3_filter = filter(lambda x: x % 3 == 0, range(1_000_000_000_000))

And the list comprehension solution, which will stall because it tries to do everything all at once (you'll need to interrupt the kernel).

In [ ]:
mult_3_comp2 = [x for x in range(1_000_000_000_000) if x % 3 == 0]

# List Comprehensions, `map`, and `filter`: Which Should You Use?
- List comprehensions are usually the easiest to read
- Other languages have features like `map` and `filter`, so you will see that from people coming in from outside python
- The cases where `map` and `filter` are necessary are not common, but you should be aware of them